In [9]:
!pip install gspread
!pip install oauth2client

In [10]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [11]:
import gspread
from google.colab import auth
import pandas as pd
import math

In [12]:

# Load the Google Sheet
sheet_url = 'https://docs.google.com/spreadsheets/d/1Yi_xPdR-ioa9HpkYFU7lW4bcarz3OZahH-91xMiXbJI/edit?usp=sharing'
spreadsheet = gc.open_by_url(sheet_url)

In [17]:

worksheet = gc.open_by_url(sheet_url).sheet1

# Get all records from the sheet
records = worksheet.get_all_records()

# Convert records to DataFrame
ndf = pd.DataFrame(records)
df = ndf.iloc[:, 2:]  # Assuming the first two columns are Timestamp and Name

# Define mapping for converting responses to numeric values
response_mapping = {
    'Strongly Disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly Agree': 5
}

# Define function to convert response to numeric value
def convert_response_to_numeric(response):
    return response_mapping.get(response, 0)

# Define function to handle reverse scoring
def handle_reverse_scoring(score, reverse):
    return 6 - score if reverse else score

# Define scales
domain_scales = {
    'Extraversion': ['1', '6', '11R', '16R', '21', '26R', '31R', '36R', '41', '46', '51R', '56'],
    'Agreeableness': ['2', '7', '12R', '17R', '22R', '27', '32', '37R', '42R', '47R', '52', '57'],
    'Conscientiousness': ['3R', '8R', '13', '18', '23R', '28R', '33', '38', '43', '48R', '53', '58R'],
    'Negative Emotionality': ['4R', '9R', '14', '19', '24R', '29R', '34', '39', '44R', '49R', '54', '59'],
    'Open-Mindedness': ['5R', '10', '15', '20', '25R', '30R', '35', '40', '45R', '50R', '55R', '60']
}

facet_scales = {
    'Sociability': ['1', '16R', '31R', '46'],
    'Assertiveness': ['6', '21', '36R', '51R'],
    'Energy Level': ['11R', '26R', '41', '56'],
    'Compassion': ['2', '17R', '32', '47R'],
    'Respectfulness': ['7', '22R', '37R', '52'],
    'Trust': ['12R', '27', '42R', '57'],
    'Organization': ['3R', '18', '33', '48R'],
    'Productiveness': ['8R', '23R', '38', '53'],
    'Responsibility': ['13', '28R', '43', '58R'],
    'Anxiety': ['4R', '19', '34', '49R'],
    'Depression': ['9R', '24R', '39', '54'],
    'Emotional Volatility': ['14', '29R', '44R', '59'],
    'Intellectual Curiosity': ['10', '25R', '40', '55R'],
    'Aesthetic Sensitivity': ['5R', '20', '35', '50R'],
    'Creative Imagination': ['15', '30R', '45R', '60']
}

# Define function to calculate sum for scales
def calculate_scale_sum(row, scales):
    scale_sum = {}
    for scale, questions in scales.items():
        total = 0
        for question in questions:
            question_index = int(question[:-1]) - 1 if question.endswith('R') else int(question) - 1
            reverse = question.endswith('R')
            response = row.iloc[question_index]
            score = convert_response_to_numeric(response)
            score = handle_reverse_scoring(score, reverse)
            total += score
        scale_sum[scale] = total
    return scale_sum

# Define function to map numeric scores to categories
def map_to_category(score):
    if score == 1:
        return "Very Low"
    elif score == 2:
        return "Low"
    elif score == 3:
        return "Medium"
    elif score == 4:
        return "High"
    elif score == 5:
        return "Very High"
    else:
        return "Unknown"
def custom_round(x):
    """Custom rounding function to round halfway values up"""
    integer_part = int(x)
    decimal_part = x - integer_part
    if decimal_part >= 0.5:
        return integer_part + 1
    else:
        return round(x)

# Process each row to calculate domain and facet scale categories
results = []
for index, row in df.iterrows():
    domain_sums = calculate_scale_sum(row, domain_scales)
    domain_avgs = {scale: total / 12 for scale, total in domain_sums.items()}
    domain_categories = {scale: map_to_category(custom_round(avg)) for scale, avg in domain_avgs.items()}

    facet_sums = calculate_scale_sum(row, facet_scales)
    facet_avgs = {scale: total / 4 for scale, total in facet_sums.items()}
    facet_categories = {scale: map_to_category(custom_round(avg)) for scale, avg in facet_avgs.items()}

    result = {**domain_categories, **facet_categories}
    result['Name:'] = ndf.loc[index, 'Name:']
    results.append(result)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Reorder columns
columns = list(results_df.columns)
columns.remove('Name:')
columns.insert(0, 'Name:')
results_df = results_df[columns]

# Print results
results_df.tail()


,Name:,Extraversion,Agreeableness,Conscientiousness,Negative Emotionality,Open-Mindedness,Sociability,Assertiveness,Energy Level,Compassion,...,Trust,Organization,Productiveness,Responsibility,Anxiety,Depression,Emotional Volatility,Intellectual Curiosity,Aesthetic Sensitivity,Creative Imagination
2,Dragon,Medium,Very High,High,Low,Very High,Low,Medium,High,Very High,...,Very High,High,High,High,Medium,Low,Very Low,Very High,High,Very High
3,Shadman,Medium,High,High,High,High,Medium,High,Medium,High,...,High,High,High,High,High,Medium,High,High,Medium,High
4,Ashraf,High,Medium,Medium,Medium,Medium,Medium,High,High,High,...,High,High,High,Medium,Medium,Medium,Medium,Medium,High,Medium
5,Saif Saruwar,Low,Medium,Low,High,Medium,Medium,Very Low,High,Medium,...,Medium,Low,Low,Medium,High,High,Medium,High,Medium,Medium
6,Asad Jashim,Very High,High,Medium,Low,Medium,Very High,High,Very High,High,...,High,Medium,High,High,Low,Very Low,Medium,Medium,Medium,High


In [14]:
# Append results_df to the second sheet
worksheet2 = spreadsheet.get_worksheet(1)  # Access the second sheet
worksheet2.clear()  # Clear the sheet first if you want to overwrite the content

# Add the DataFrame to the sheet
worksheet2.update([results_df.columns.values.tolist()] + results_df.values.tolist())

{'spreadsheetId': '1Yi_xPdR-ioa9HpkYFU7lW4bcarz3OZahH-91xMiXbJI',
 'updatedRange': 'Results!A1:U8',
 'updatedRows': 8,
 'updatedColumns': 21,
 'updatedCells': 168}

In [15]:
name_ = input()

Asad Jashim 


In [16]:
import pandas as pd
from IPython.display import display, HTML

# Load the CSV file
#file_path = '/content/drive/MyDrive/show3.csv'
data = results_df


# Define the fixed set of 20 columns
fixed_columns = [
    'Agreeableness', 'Conscientiousness', 'Open-Mindedness', 'Compassion',
    'Respectfulness', 'Trust', 'Organization', 'Productiveness',
    'Responsibility', 'Intellectual Curiosity', 'Aesthetic Sensitivity',
    'Creative Imagination', 'Extraversion', 'Assertiveness', 'Energy Level',
    'Anxiety', 'Negative Emotionality', 'Sociability', 'Depression',
    'Emotional Volatility'
]

# Define the descriptions for each level of each trait
descriptions = {
    "Openness": {
        "Very Low": "prefers routine and traditional",
        "Low": "somewhat conventional and slightly resistant to change",
        "Medium": "balanced between openness and conventionality",
        "High": "creative and open to new experiences",
        "Very High": "very creative and also imaginative and open to new experiences"
    },
    "Conscientiousness": {
        "Very Low": "disorganized and spontaneous",
        "Low": "somewhat unreliable and prefers flexibility",
        "Medium": "moderately organized and dependable",
        "High": "organized and reliable",
        "Very High": "highly organized and disciplined"
    },
    "Extraversion": {
        "Very Low": "very reserved and prefers solitude",
        "Low": "somewhat reserved and enjoys some solitary activities",
        "Medium": "balanced between sociability and reserved behavior",
        "High": "outgoing and sociable",
        "Very High": "highly outgoing also energetic and sociable"
    },
    "Agreeableness": {
        "Very Low": "competitive and less cooperative",
        "Low": "somewhat self-centered and less empathetic",
        "Medium": "average in cooperation and compassion",
        "High": "cooperative and compassionate",
        "Very High": "highly cooperative also kind and compassionate"
    },
    "Neuroticism": {
        "Very Low": "very emotionally stable and calm",
        "Low": "generally calm and less prone to stress",
        "Medium": "average in emotional stability",
        "High": "somewhat prone to stress, more sensitive",
        "Very High": "prone to stress and emotional instability"
    }
}

# Function to generate the personality profile
def generate_profile(O, C, E, A, N):
    O_desc = descriptions["Openness"][O]
    C_desc = descriptions["Conscientiousness"][C]
    E_desc = descriptions["Extraversion"][E]
    A_desc = descriptions["Agreeableness"][A]
    N_desc = descriptions["Neuroticism"][N]

    profile = (
        f"<h2>Personality Profile</h2>"
        f"<p><strong>Openness:</strong> {O_desc.capitalize()}</p>"
        f"<p><strong>Conscientiousness:</strong> {C_desc.capitalize()}</p>"
        f"<p><strong>Extraversion:</strong> {E_desc.capitalize()}</p>"
        f"<p><strong>Agreeableness:</strong> {A_desc.capitalize()}</p>"
        f"<p><strong>Neuroticism:</strong> {N_desc.capitalize()}</p>"
    )

    summary = (
        f"<h3>Summary</h3>"
        f"<p>This individual is likely to be {O_desc.split(',')[0]} also {C_desc.split(',')[0]}. "
        f"They may be {E_desc.split(',')[0]} and {A_desc.split(',')[0]}. They handle stress {N_desc.split(',')[0]}.</p>"
    )

    return profile, summary

# Function to extract traits for a specific name and categorize them
def extract_traits(name):
    # Filter the dataframe for the given name
    person_data = data[data['Name:'] == name]

    if person_data.empty:
        return f"No data found for {name}."

    # Ensure only the fixed set of columns are considered
    person_data = person_data[fixed_columns]

    # Transpose the dataframe to switch rows and columns for easier processing
    traits = person_data.T
    traits.columns = ['Value']

    # Categorize traits based on their values
    categories = {
        'Very High': [],
        'High': [],
        'Medium': [],
        'Low': [],
        'Very Low': []
    }

    for trait, value in traits['Value'].items():
        if value == 'Very High':
            categories['Very High'].append(trait)
        elif value == 'High':
            categories['High'].append(trait)
        elif value == 'Medium':
            categories['Medium'].append(trait)
        elif value == 'Low':
            categories['Low'].append(trait)
        elif value == 'Very Low':
            categories['Very Low'].append(trait)

    # Create an HTML table for categorized traits
    table_html = "<h2>Traits for {}</h2>".format(name)
    for category, traits_list in categories.items():
        table_html += "<h3>{} ({})</h3><ul>".format(category, len(traits_list))
        for trait in traits_list:
            table_html += "<li>{}</li>".format(trait)
        table_html += "</ul>"

    # Map traits to the OCEAN model
    O = traits.loc[['Open-Mindedness', 'Intellectual Curiosity', 'Aesthetic Sensitivity', 'Creative Imagination']].iloc[:, 0].mode()[0]
    C = traits.loc[['Conscientiousness', 'Organization', 'Productiveness', 'Responsibility']].iloc[:, 0].mode()[0]
    E = traits.loc[['Extraversion', 'Assertiveness', 'Energy Level', 'Sociability']].iloc[:, 0].mode()[0]
    A = traits.loc[['Agreeableness', 'Compassion', 'Respectfulness', 'Trust']].iloc[:, 0].mode()[0]
    N = traits.loc[['Anxiety', 'Negative Emotionality', 'Depression', 'Emotional Volatility']].iloc[:, 0].mode()[0]

    # Generate and display the personality profile
    profile, summary = generate_profile(O, C, E, A, N)

    # Define subdomains for each main trait
    subdomains = {
        "Openness": ["Aesthetic Sensitivity", "Intellectual Curiosity", "Creative Imagination"],
        "Conscientiousness": ["Organization", "Productiveness", "Responsibility"],
        "Extraversion": ["Sociability", "Assertiveness", "Energy Level"],
        "Agreeableness": ["Compassion", "Respectfulness", "Trust"],
        "Neuroticism": ["Anxiety", "Depression", "Emotional Volatility"]
    }

    # Collect the main traits and their subdomains with values
    main_traits = {
        "Openness": O,
        "Conscientiousness": C,
        "Extraversion": E,
        "Agreeableness": A,
        "Neuroticism": N
    }

    # Generate HTML for the main traits and their subdomains
    main_traits_html = (
        "<h2>Main Traits and Subdomains</h2>"
        "<table border='1' style='border-collapse: collapse; width: 100%;'>"
        "<tr><th>Main Trait</th><th>Level</th><th>Subdomains</th><th>Subdomain Levels</th></tr>"
    )
    for trait, level in main_traits.items():
        subdomain_names = subdomains[trait]
        subdomain_levels = [traits.loc[subdomain, 'Value'] for subdomain in subdomain_names]
        main_traits_html += (
            f"<tr>"
            f"<td><strong>{trait}</strong></td>"
            f"<td>{level}</td>"
            f"<td>{', '.join(subdomain_names)}</td>"
            f"<td>{', '.join(subdomain_levels)}</td>"
            f"</tr>"
        )
    main_traits_html += "</table>"

    # Display the table and profile using HTML
    display(HTML(table_html))
    display(HTML(main_traits_html))
    display(HTML(profile))
    display(HTML(summary))

# Example usage
name_to_search = name_
extract_traits(name_to_search)


Main Trait,Level,Subdomains,Subdomain Levels
Openness,Medium,"Aesthetic Sensitivity, Intellectual Curiosity, Creative Imagination","Medium, Medium, High"
Conscientiousness,High,"Organization, Productiveness, Responsibility","Medium, High, High"
Extraversion,Very High,"Sociability, Assertiveness, Energy Level","Very High, High, Very High"
Agreeableness,High,"Compassion, Respectfulness, Trust","High, High, High"
Neuroticism,Low,"Anxiety, Depression, Emotional Volatility","Low, Very Low, Medium"
